# Assignment 1: Logistic Regression
Welcome to week one of this specialization. You will learn about logistic regression. Concretely, you will be implementing logistic regression for sentiment analysis on tweets. Given a tweet, you will decide if it has a positive sentiment or a negative one. Specifically you will: 

* Learn how to extract features for logistic regression given some text
* Implement logistic regression from scratch
* Apply logistic regression on a natural language processing task
* Test using your logistic regression
* Perform error analysis

We will be using a data set of tweets. Hopefully you will get more than 99% accuracy.  
Run the cell below to load in the packages.

## Import functions and data

In [1]:
# run this cell to import nltk
import nltk
from os import getcwd

### Imported functions

Download the data needed for this assignment. Check out the [documentation for the twitter_samples dataset](http://www.nltk.org/howto/twitter.html).

* twitter_samples: if you're running this notebook on your local computer, you will need to download it using:
```Python
nltk.download('twitter_samples')
```

* stopwords: if you're running this notebook on your local computer, you will need to download it using:
```python
nltk.download('stopwords')
```

#### Import some helper functions that we provided in the utils.py file:
* `process_tweet()`: cleans the text, tokenizes it into separate words, removes stopwords, and converts words to stems.
* `build_freqs()`: this counts how often a word in the 'corpus' (the entire set of tweets) was associated with a positive label '1' or a negative label '0', then builds the `freqs` dictionary, where each key is a (word,label) tuple, and the value is the count of its frequency within the corpus of tweets.

In [2]:
# add folder, tmp2, from our local workspace containing pre-downloaded corpora files to nltk's data path
# this enables importing of these files without downloading it again when we refresh our workspace

filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [3]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 

from utils import process_tweet, build_freqs

### Prepare the data
* The `twitter_samples` contains subsets of 5,000 positive tweets, 5,000 negative tweets, and the full set of 10,000 tweets.  
    * If you used all three datasets, we would introduce duplicates of the positive tweets and negative tweets.  
    * You will select just the five thousand positive tweets and five thousand negative tweets.

In [4]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

* Train test split: 20% will be in the test set, and 80% in the training set.


In [5]:
# split the data into two pieces, one for training and one for testing (validation set) 
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

* Create the numpy array of positive labels and negative labels.

In [6]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [7]:
# Print the shape train and test sets
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


* Create the frequency dictionary using the imported `build_freqs()` function.  
    * We highly recommend that you open `utils.py` and read the `build_freqs()` function to understand what it is doing.
    * To view the file directory, go to the menu and click File->Open.

```Python
    for y,tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
```
* Notice how the outer for loop goes through each tweet, and the inner for loop steps through each word in a tweet.
* The `freqs` dictionary is the frequency dictionary that's being built. 
* The key is the tuple (word, label), such as ("happy",1) or ("happy",0).  The value stored for each key is the count of how many times the word "happy" was associated with a positive label, or how many times "happy" was associated with a negative label.

In [8]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11346


#### Expected output
```
type(freqs) = <class 'dict'>
len(freqs) = 11346
```

### Process tweet
The given function `process_tweet()` tokenizes the tweet into individual words, removes stop words and applies stemming.

In [9]:
# test the function below
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

This is an example of the processed version of the tweet: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


#### Expected output
```
This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
 
This is an example of the processes version: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']
```

# Part 1: Logistic regression 


### Part 1.1: Sigmoid
You will learn to use logistic regression for text classification. 
* The sigmoid function is defined as: 

$$ h(z) = \frac{1}{1+\exp^{-z}} \tag{1}$$

It maps the input 'z' to a value that ranges between 0 and 1, and so it can be treated as a probability. 

<div style="width:image width px; font-size:100%; text-align:center;"><img src='../tmp2/sigmoid_plot.jpg' alt="alternate text" width="width" height="height" style="width:300px;height:200px;" /> Figure 1 </div>

#### Instructions: Implement the sigmoid function
* You will want this function to work if z is a scalar as well as if it is an array.

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li><a href="https://docs.scipy.org/doc/numpy/reference/generated/numpy.exp.html" > numpy.exp </a> </li>

</ul>
</p>



In [10]:
# UNQ_C1 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def sigmoid(z): 
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    # calculate the sigmoid of z
    h = 1/(1+ np.exp(-z))
    ### END CODE HERE ###
    
    return h

In [11]:
# Testing your function 
if (sigmoid(0) == 0.5):
    print('SUCCESS!')
else:
    print('Oops!')

if (sigmoid(4.92) == 0.9927537604041685):
    print('CORRECT!')
else:
    print('Oops again!')

SUCCESS!
CORRECT!


### Logistic regression: regression and a sigmoid

Logistic regression takes a regular linear regression, and applies a sigmoid to the output of the linear regression.

Regression:
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
Note that the $\theta$ values are "weights". If you took the Deep Learning Specialization, we referred to the weights with the `w` vector.  In this course, we're using a different variable $\theta$ to refer to the weights.

Logistic regression
$$ h(z) = \frac{1}{1+\exp^{-z}}$$
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
We will refer to 'z' as the 'logits'.

### Part 1.2 Cost function and Gradient

The cost function used for logistic regression is the average of the log loss across all training examples:

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)}))\tag{5} $$
* $m$ is the number of training examples
* $y^{(i)}$ is the actual label of the i-th training example.
* $h(z(\theta)^{(i)})$ is the model's prediction for the i-th training example.

The loss function for a single training example is
$$ Loss = -1 \times \left( y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)})) \right)$$

* All the $h$ values are between 0 and 1, so the logs will be negative. That is the reason for the factor of -1 applied to the sum of the two loss terms.
* Note that when the model predicts 1 ($h(z(\theta)) = 1$) and the label $y$ is also 1, the loss for that training example is 0. 
* Similarly, when the model predicts 0 ($h(z(\theta)) = 0$) and the actual label is also 0, the loss for that training example is 0. 
* However, when the model prediction is close to 1 ($h(z(\theta)) = 0.9999$) and the label is 0, the second term of the log loss becomes a large negative number, which is then multiplied by the overall factor of -1 to convert it to a positive loss value. $-1 \times (1 - 0) \times log(1 - 0.9999) \approx 9.2$ The closer the model prediction gets to 1, the larger the loss.

In [12]:
# verify that when the model predicts close to 1, but the actual label is 0, the loss is a large positive value
-1 * (1 - 0) * np.log(1 - 0.9999) # loss is about 9.2

9.210340371976294

* Likewise, if the model predicts close to 0 ($h(z) = 0.0001$) but the actual label is 1, the first term in the loss function becomes a large number: $-1 \times log(0.0001) \approx 9.2$.  The closer the prediction is to zero, the larger the loss.

In [13]:
# verify that when the model predicts close to 0 but the actual label is 1, the loss is a large positive value
-1 * np.log(0.0001) # loss is about 9.2

9.210340371976182

#### Update the weights

To update your weight vector $\theta$, you will apply gradient descent to iteratively improve your model's predictions.  
The gradient of the cost function $J$ with respect to one of the weights $\theta_j$ is:

$$\nabla_{\theta_j}J(\theta) = \frac{1}{m} \sum_{i=1}^m(h^{(i)}-y^{(i)})x^{(i)}_j \tag{5}$$
* 'i' is the index across all 'm' training examples.
* 'j' is the index of the weight $\theta_j$, so $x^{(i)}_j$ is the feature associated with weight $\theta_j$

* To update the weight $\theta_j$, we adjust it by subtracting a fraction of the gradient determined by $\alpha$:
$$\theta_j = \theta_j - \alpha \times \nabla_{\theta_j}J(\theta) $$
* The learning rate $\alpha$ is a value that we choose to control how big a single update will be.



## Instructions: Implement gradient descent function
* The number of iterations `num_iters` is the number of times that you'll use the entire training set.
* For each iteration, you'll calculate the cost function using all training examples (there are `m` training examples), and for all features.
* Instead of updating a single weight $\theta_i$ at a time, we can update all the weights in the column vector:  
$$\mathbf{\theta} = \begin{pmatrix}
\theta_0
\\
\theta_1
\\ 
\theta_2 
\\ 
\vdots
\\ 
\theta_n
\end{pmatrix}$$
* $\mathbf{\theta}$ has dimensions (n+1, 1), where 'n' is the number of features, and there is one more element for the bias term $\theta_0$ (note that the corresponding feature value $\mathbf{x_0}$ is 1).
* The 'logits', 'z', are calculated by multiplying the feature matrix 'x' with the weight vector 'theta'.  $z = \mathbf{x}\mathbf{\theta}$
    * $\mathbf{x}$ has dimensions (m, n+1) 
    * $\mathbf{\theta}$: has dimensions (n+1, 1)
    * $\mathbf{z}$: has dimensions (m, 1)
* The prediction 'h', is calculated by applying the sigmoid to each element in 'z': $h(z) = sigmoid(z)$, and has dimensions (m,1).
* The cost function $J$ is calculated by taking the dot product of the vectors 'y' and 'log(h)'.  Since both 'y' and 'h' are column vectors (m,1), transpose the vector to the left, so that matrix multiplication of a row vector with column vector performs the dot product.
$$J = \frac{-1}{m} \times \left(\mathbf{y}^T \cdot log(\mathbf{h}) + \mathbf{(1-y)}^T \cdot log(\mathbf{1-h}) \right)$$
* The update of theta is also vectorized.  Because the dimensions of $\mathbf{x}$ are (m, n+1), and both $\mathbf{h}$ and $\mathbf{y}$ are (m, 1), we need to transpose the $\mathbf{x}$ and place it on the left in order to perform matrix multiplication, which then yields the (n+1, 1) answer we need:
$$\mathbf{\theta} = \mathbf{\theta} - \frac{\alpha}{m} \times \left( \mathbf{x}^T \cdot \left( \mathbf{h-y} \right) \right)$$

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>use np.dot for matrix multiplication.</li>
    <li>To ensure that the fraction -1/m is a decimal value, cast either the numerator or denominator (or both), like `float(1)`, or write `1.` for the float version of 1. </li>
</ul>
</p>



In [14]:
# UNQ_C2 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    # get 'm', the number of rows in matrix x
    print (x.shape)
    m = x.shape[0] #x.shape([0])
    
    for i in range(0, num_iters):
        
        # get z, the dot product of x and theta
        z = np.dot(x,theta)
        
        # get the sigmoid of z
        h = sigmoid(z)
        
        # calculate the cost function
        J = (-1/m) * (np.dot(y.T,np.log(h)) + np.dot((1-y).T,np.log(1-h)))

        # update the weights theta
        theta = theta - (alpha/m) * np.dot(x.T,(h-y))
        
    ### END CODE HERE ###
    J = float(J)
    return J, theta

In [15]:
# Check the function
# Construct a synthetic test case using numpy PRNG functions
np.random.seed(1)
# X input is 10 x 3 with ones for the bias terms
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
# Y Labels are 10 x 1
tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)

# Apply gradient descent
tmp_J, tmp_theta = gradientDescent(tmp_X, tmp_Y, np.zeros((3, 1)), 1e-8, 700)
print(f"The cost after training is {tmp_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(tmp_theta)]}")

(10, 3)
The cost after training is 0.67094970.
The resulting vector of weights is [4.1e-07, 0.00035658, 7.309e-05]


#### Expected output
```
The cost after training is 0.67094970.
The resulting vector of weights is [4.1e-07, 0.00035658, 7.309e-05]
```

## Part 2: Extracting the features

* Given a list of tweets, extract the features and store them in a matrix. You will extract two features.
    * The first feature is the number of positive words in a tweet.
    * The second feature is the number of negative words in a tweet. 
* Then train your logistic regression classifier on these features.
* Test the classifier on a validation set. 

### Instructions: Implement the extract_features function. 
* This function takes in a single tweet.
* Process the tweet using the imported `process_tweet()` function and save the list of tweet words.
* Loop through each word in the list of processed words
    * For each word, check the `freqs` dictionary for the count when that word has a positive '1' label. (Check for the key (word, 1.0)
    * Do the same for the count for when the word is associated with the negative label '0'. (Check for the key (word, 0.0).)


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>Make sure you handle cases when the (word, label) key is not found in the dictionary. </li>
    <li> Search the web for hints about using the `.get()` method of a Python dictionary.  Here is an <a href="https://www.programiz.com/python-programming/methods/dictionary/get" > example </a> </li>
</ul>
</p>


In [50]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # loop through each word in the list of words
    for word in word_l:
        print (word)
        
        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word, 1.0),0) #freqs[(word,1.0)]
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word, 0),0) #freqs[(word,0)]
        
    ### END CODE HERE ###
    assert(x.shape == (1, 3))
    return x

In [51]:
# Check your function

# test 1
# test on training data
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

followfriday
top
engag
member
commun
week
:)
[[1.00e+00 3.02e+03 6.30e+01]]


#### Expected output
```
[[1.00e+00 3.02e+03 6.10e+01]]
```

In [52]:
# test 2:
# check for when the words are not in the freqs dictionary
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

blorb
bleeeb
blooob
[[1. 3. 3.]]


#### Expected output
```
[[1. 0. 0.]]
```

## Part 3: Training Your Model

To train the model:
* Stack the features for all training examples into a matrix `X`. 
* Call `gradientDescent`, which you've implemented above.

This section is given to you.  Please read it for understanding and run the cell.

In [53]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

followfriday
top
engag
member
commun
week
:)
hey
jame
odd
:/
pleas
call
contact
centr
02392441234
abl
assist
:)
mani
thank
listen
last
night
:)
bleed
amaz
track
scotland
congrat
:)
yeaaah
yipppi
accnt
verifi
rqst
succeed
got
blue
tick
mark
fb
profil
:)
15
day
one
irresist
:)
flipkartfashionfriday
like
keep
love
custom
wait
long
hope
enjoy
happi
friday
lwwf
:)
second
thought
’
enough
time
dd
:)
new
short
enter
system
sheep
must
buy
jgh
go
bayan
:D
bye
act
mischiev
call
etl
layer
in-hous
wareh
app
katamari
well
…
name
impli
:p
followfriday
top
influenc
commun
week
:)
love
big
...
juici
...
selfi
:)
follow
perfect
alreadi
know
what'
wait
:)
great
new
opportun
junior
triathlet
age
12
13
gatorad
seri
get
entri
:)
lay
greet
card
rang
print
today
love
job
:-)
friend'
lunch
...
yummm
:)
nostalgia
tb
ku
id
conflict
thank
help
:D
here'
screenshot
work
hi
liv
:)
hello
need
know
someth
u
fm
twitter
—
sure
thing
:)
dm
x
followfriday
top
new
follow
commun
week
:)
i'v
heard
four
season
pretti
dope
pe

never
see
posit
kha
u
could
also
mention
atleast
go
:)
hi
bam
follow
bestfriend
love
lot
:)
see
warsaw
<3
love
<3
x39
mango
lassi'
monty'
marvel
:D
though
suspect
that'
meant
...
followfriday
top
influenc
commun
week
:)
haha
ye
24
hr
time
come
touch
kepler
452b
chalna
hai
:D
follow
thankyou
pretti
:)
definit
hazel
food
market
brooklyn
pta
:)
still
awak
:)
okayi
:D
awww
thank
:)
ha
surpris
least
okay
see
doc
afternoon
yorkshir
splendid
though
:)
xx
spam
folder
wow
huge
amount
need
travel
nigeria
claim
:)
rted
good
luck
:)
leg
hurt
bad
:)
get
mine
saturday
:-)
thaaank
decemb
puhon
:)
happinesss
:)
true
chang
tnc
without
prior
notif
could
use
last
night
:-)
fat
co
probabl
ate
yuna
:D
:D
:D
fun
tamesid
:)
:-)
:-)
:-)
´
welcom
:)
googl
account
:)
call
scouser
everyth
hope
zoe
well
mate
:)
right
liter
like
mayb
u
get
peopl
actual
know
they'r
:-)
samee
rest
well
:)
happi
birthday
edgar
hope
good
one
keep
updat
travel
log
:)
hello
:D
bring
abe
:)
stat
day
arriv
2
new
follow
unfollow
:)
via
som

hi
im
interest
internship
program
sadli
cant
access
career
page
u
help
issu
:)
sad
hear
thank
much
overwhelmingli
beauti
game
thank
hard
work
:)
aha
yesss
look
forward
see
:)
happi
friday
beaut
:)
♬
ff
love
help
would
love
win
deo
faaabul
:D
freebiefriday
aluminiumfre
stayfresh
xx
ye
john
:)
thank
worri
i'll
navig
blog
check
:)
thnk
progrmr
:-)
bet
well
get
sleep
9pm
shift
9am
quit
hard
work
thank
:)
xx
figur
:-)
hardli
surpris
rose
inspir
much
emot
poetri
:)
follow
:)
join
frequentfly
program
:)
alway
end
break
apolog
fuck
nice
:-)
fabul
well
hope
great
day
:-)
kb
hi
would
like
concert
let
know
citi
countri
i'll
start
work
thank
:)
welcom
:)
visit
blog
enjoy
londondairi
icecream
share
us
experi
:)
follow
luv
ur
:-)
past
hoodi
cover
lot
sin
:)
bet
excit
:)
:')
xxx
good
luck
hope
get
number
one
welcom
jim
made
chuckl
train
morn
:-)
happi
friday
shop
bit
:p
agre
haha
:p
happi
birthday
we'r
posit
full
cake
:)
doh
500
subscrib
need
:)
pleas
let
us
know
reach
number
i'm
good
thank
hope
well

mm
okay
thank
:)
messag
us
right
nicola
forward
:)
brighten
day
:D
helen
follow
congrat
:)
hi
brian
i'll
get
ship
today
take
2-3
week
australia
mani
thank
:)
good
fun
time
ol
bone
creak
:)
eh
abuti
welcom
tweetland
...
lol
:p
thx
android
support
xma
:)
great
skyblock
stream
:)
im
stand
alway
belieb
:)
bcaus
2009
die
:)
stream
twitch
join
:D
..
sympathi
laugh
:)
would
like
:)
xo
awesom
:D
unniee
:)
tonight
night
...
:)
nuka
penacova
djset
free
edm
kizomba
latinhous
housemus
portug
2015
summer
well
wild
ride
>:d
hahaha
tell
anytim
wanna
tast
oh
yer
address
send
:)
thank
team
yellow
:)
mtn
8
maganda
look
pretti
short
hair
cute
:D
good
morn
gorgeou
mistress
saphir
:)
i'm
work
i'm
busi
help
finish
im
home
:)
thank
amaz
4000
follow
instagram
´
among
yet
feel
free
connect
:-)
...
coconut
sambal
mussel
recip
:-)
thank
kalin
:)
follow
mixcloud
:)
fback
:)
thankyou
greet
australia
:)
cant
work
sarcasm
:)
know
chelsea
support
think
he
useless
come
ok
follow
back
:)
come
back
next
thursday
still
h

es
mi
tio
:)
know
must
follow
sinc
i'm
sinaloa
..
arr
must
hahaha
:p
realli
stylish
trendi
good
go
kim
keep
:)
love
fabfriday
:)
facetim
calum
weekend
constantli
messag
luke
:)
follow
well
thank
guy
sure
thing
hope
ok
:)
info
yet
we'll
announc
soon
get
filbarbarian
:)
i'v
beer
talk
i'd
say
2
broken
arm
issu
1
testicl
light
:)
proud
:D
:D
night
social
app
katerina
maniataki
design
idea
ahh
alright
:)
wanna
know
worthwhil
judg
tech
review
better
window
8
found
later
sorri
ask
stupid
question
love
plugin
far
also
love
bass
slap
ig
play
:)
6pm
door
open
everyon
vip
gener
ticket
get
seat
earli
watch
support
act
:)
fback
:)
london
babi
:)
toptravelcentar
ttctop
lux
luxurytravel
beograd
srbija
travel
putovanja
…
thank
wendi
provid
fresh
water
area
provid
drainag
servic
ami
:)
homebound
:D
hahahay
welcom
:D
yeeeah
:D
…
moar
kitteh
incom
:D
look
forward
meet
tower
today
:)
yippee
hope
littl
one
enjoy
scrummi
recip
:)
ur
bio
move
mcpe
->
vainglori
:)
u
one
day
wish
:)
thank
:)
awesom
driver
6:01

...

hay
:(
dont
know
:(
:(
okay
babi
angel
die
:(
xue
made
sing
song
long
D:
alway
give
midori
work
:(
hi
pleas
help
get
back
disabl
account
realli
realli
need
back
:(
ang
sakit
insid
:(
alreadi
:(
follow
thank
pleas
follow
:(
:(
disgust
piec
shit
..
amaz
mother
mateo
brooklyn
lucki
💗
deserv
guy
buy
u
water
u
can't
drink
u
watch
cup
like
hawk
transfer
bartend
u
:(
:(
teenchoic
choiceinternationalartist
superjunior
work
home
normal
mean
surf
case
:(
:(
...
feel
better
bbi
ok
better
stay
otherwis
miss
much
:(
that'
shock
can't
peopl
live
let
live
sometim
despair
world
:(
hope
ok
follow
thank
pleas
follow
:(
bc
still
hope
:(
hell
insta
work
:(
notic
:(
miss
chri
voic
:(
hi
may
request
promo
code
pl
tri
use
iwantin
got
reject
:(
:(
pl
think
good
enough
___
let
he'
miss
chanc
fault
omfg
harri
pl
bae
:(
ignor
follow
thank
pleas
follow
:(
goodluck
pocket
i'm
move
next
week
:(
month
sinc
saw
:(
realli
sorri
receiv
wrong
order
:(
would
like
sort
straight
away
email
help@veryhq.co.uk
photo
benedicte

hey
i'v
receiv
lootcrat
today
box
damag
damag
item
insid
:(
hard
pick
one
inca
trail
tough
fair
sandboard
fun
lost
iphon
:(
much
:(
especi
tv
network
cancel
time
:'(
ice
cream
avail
surpris
...
leed
:(
cri
derbi
counti
leed
unit
coffe
mug
unabl
share
hate
man
u
:(
follow
thank
pleas
follow
:(
yeah
true
fuck
one
signatur
dish
last
night
unfamiliar
kitchen
:(
hate
that'
sure
:(
bad
news
noth
new
wait
coldest
weekend
load
shed
:(
10
year
old'
cat
first
loss
pet
:(
thank
someon
mayb
look
case
14518344
61
fast
total
fuck
mad
expens
:(
thirdwheel
lovebird
:(
cri
nth
time
today
:(
kinda
tire
tho
hay
gonna
life
:(
dark
hors
better
imo
sorri
sorri
dont
cri
babi
talk
:(
:(
mean
:(
still
lot
tweet
delet
hand
...
:(
dude
even
go
watch
:(
feel
like
shirt
familiar
:(
truth
could
anyon
would
still
choos
>:(
:(
ahhh
last
2
stage
go
:(
account
got
hack
:(
@juliettemaughan
realis
never
receiv
copi
sensiesha
:(
tri
think
return
spain
leav
eldest
behind
alreadi
one
cri
:(
miss
netbal
:(
omg
u
know
...
😟
h

tri
get
field
goat
jump
wall
drop
low
:(
iran
irand
:-(
us-iran
nuclear
deal
mit'
expert
size
deal
sever
li
...
can't
get
anywher
:-(
sorri
god
:(
s2e12
probabl
saddest
ive
ever
felt
rumpi
:-(
daddi
chang
channel
:(
liter
ate
whole
gallon
ice
cream
idk
proud
nah
:(
ryan
ah
guy
know
thing
weather
set
sleep
respons
:-(
secret
:(
come
dalla
dandia
right
:(
rbi
make
cage
parrot
1li
elect
commiss
cag
r
left
power
stripe
gujarat
model
complet
sad
:(
i'm
look
forward
mine
goe
next
year
3
:(
kind
eye
tear
tear
hard
care
:(
ily.melani
aww
:(
lol
realli
unlik
:(
pure
talent
:(
alway
fan
deepxcap
brother
talent
�
�
�
doin
ok
:(
want
go
cannot
:(
cant
go
sleep
:-(
5:08
:-(
’
thesi
:(
miss
u
:-(
sadli
must
:(
agre
:(
❤
❤
❤
like
boy
like
laugh
hug
said
bye
:(
love
pleas
belieb
stop
vote
twitter
count
:(
u
vote
come
ubericecream
...
car
:(
london
summer
snapchat
tammirossm
snapchat
kikgirl
kikchat
wet
wife
indiemus
sexi
:(
we'll
never
young
...
young
:(
wish
london
:(
gonna
stay
airport
bit
go
citi
g

read
sandrabland
stori
that
scari
hope
truth
reveal
:(
aw
:(
make
monday
nawww
hang
love
:(
ye
ye
chikka
ug
kita
rata
soonest
mwamwa
can't
get
fit
:(
faggot
:(
poor
english
teacher
listen
radio
:-(
yep
:(
exactli
id
anyth
go
thorp
park
tomorrow
:(
wanna
go
vidcon
:(
eye
open
realis
thing
like
happen
scari
bad
someon
would
:(
someon
send
screenshot
convers
want
see
phone
stupid
:-(
doubt
i'm
go
bed
soon
though
:(
wake
like
9
fyi
url
profil
work
:(
he'
sleep
he'
cute
:-(
saw
nicest
mehendi
design
someone'
pictur
dash
can't
find
bookmark
>:(
omg
whay
ur
di
:(
i'm
tell
:(
shaa
allah
na
forc
chang
phone
mana
😒
sorri
na
:(
next
time
prami
😚
school
end
earlier
could
ngee
ann
acoust
gig
:(
make
sad
go
back
:(
want
jack
box
:(
crikey
bit
goal
sound
like
someon
turn
work
snit
:(
block
peopl
make
lose
follow
:(
:(
tire
week
:(
nathanielhinanakit
naya
what
happen
link
keep
spinni
load
wheel
never
play
anyth
:(
:(
onlin
day
ago
like
50
notif
:(
2
day
dammit
stop
guy
lol
jk
<3
u
...
gosh
...
i'm
rea

evil
pepper
system
:(
kik
thessidew
877
kik
kikmeguy
facetim
omegl
bore
quot
sexi
:(
there'
meet
sunday
genesi
back
mexico
yet
:(
upset
i'm
go
novemb
man
hope
there'
go
next
time
uk
:(
crave
wing
:(
mash
phone
lost
number
..
:(
whattsap
avail
yeah
:(
idk
never
found
time
finish
inuyasha
i'v
seen
lot
anim
grew
watch
inuyasha
hey
girl
must
googl
plu
can't
convinc
fr
end
hang
outfwith
:(
want
myungsoo
gyu
hoya
yeol
:(
victoria
park
book
close
:(
hard
find
organis
we'r
satisfi
:(
god
im
sorri
hope
better
:-(
wah
stock
sa
1
hour
:(
lay
challo
pliss
:(
hell
:(
omg
i'd
facetim
juliana
rn
phone
:-(
still
confus
enrol
sem
:(
i'm
work
next
weekend
:(
:(
even
u
hit
5
fell
asleep
:(
guy
wanna
come
starbuck
w
lol
darlen
followback
pleas
:(
gay
togeth
least
leav
alon
:(
emoji
:(
freak
cat
cuddl
:-(
see
meet
greet
tour
come
unfortun
come
brisban
:(
late
:(
hi
merlin
current
wait
roof
repair
kitchen
let
water
:(
saw
mention
omg
sorri
:-(
i'm
sorri
sweeti
:(
one
deserv
nawwwe
miss
hyperbulli
like
tong


**Expected Output**: 

```
The cost after training is 0.24216529.
The resulting vector of weights is [7e-08, 0.0005239, -0.00055517]
```

# Part 4: Test your logistic regression

It is time for you to test your logistic regression function on some new input that your model has not seen before. 

#### Instructions: Write `predict_tweet`
Predict whether a tweet is positive or negative.

* Given a tweet, process it, then extract the features.
* Apply the model's learned weights on the features to get the logits.
* Apply the sigmoid to the logits to get the prediction (a value between 0 and 1).

$$y_{pred} = sigmoid(\mathbf{x} \cdot \theta)$$

In [54]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def predict_tweet(tweet, freqs, theta):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs)
    
    # make the prediction using x and theta
    y_pred = sigmoid (np.dot(x,theta))
    
    ### END CODE HERE ###
    
    return y_pred

In [55]:
# Run this cell to test your function
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))

happi
I am happy -> 0.518578
bad
I am bad -> 0.494341
movi
great
this movie should have been great. -> 0.515330
great
great -> 0.515462
great
great
great great -> 0.530895
great
great
great
great great great -> 0.546269
great
great
great
great
great great great great -> 0.561555


**Expected Output**: 
```
I am happy -> 0.518580
I am bad -> 0.494339
this movie should have been great. -> 0.515331
great -> 0.515464
great great -> 0.530898
great great great -> 0.546273
great great great great -> 0.561561
```

In [56]:
# Feel free to check the sentiment of your own tweet below
my_tweet = 'I am learning :)'
predict_tweet(my_tweet, freqs, theta)

learn
:)


array([[0.8163391]])

## Check performance using the test set
After training your model using the training set above, check how your model might perform on real, unseen data, by testing it against the test set.

#### Instructions: Implement `test_logistic_regression` 
* Given the test data and the weights of your trained model, calculate the accuracy of your logistic regression model. 
* Use your `predict_tweet()` function to make predictions on each tweet in the test set.
* If the prediction is > 0.5, set the model's classification `y_hat` to 1, otherwise set the model's classification `y_hat` to 0.
* A prediction is accurate when `y_hat` equals `test_y`.  Sum up all the instances when they are equal and divide by `m`.


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>Use np.asarray() to convert a list to a numpy array</li>
    <li>Use np.squeeze() to make an (m,1) dimensional array into an (m,) array </li>
</ul>
</p>

In [57]:
# UNQ_C5 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def test_logistic_regression(test_x, test_y, freqs, theta):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # the list for storing predictions
    y_hat = []
    
    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1)
        else:
            # append 0 to the list
            y_hat.append(0)
    print (len(y_hat))
    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator
    print (test_y.shape)
    m = test_y.shape[0]
    accuracy = (1/m) * (np.sum((y_hat == test_y.flatten())))

    ### END CODE HERE ###
    
    return accuracy

In [58]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

bro
u
wan
cut
hair
anot
ur
hair
long
liao
bo
sinc
ord
liao
take
easi
lor
treat
save
leav
longer
:)
bro
lol
sibei
xialan
back
thnx
god
i'm
happi
:)
thought
ear
malfunct
thank
good
clear
one
apolog
:-)
stuck
centr
right
clown
right
joker
left
...
:)
happi
friday
:-)
follow
:)
x
teenchoic
choiceinternationalartist
superjunior
fight
oppa
:D
birthday
today
birthday
wish
hope
there'
good
news
ben
soon
:-)
good
morn
:-)
friday
󾰀
plan
day
current
play
shop
...
happi
friday
:)
3
good
nigth
:)
estoy
escuchando
enemi
god
actual
bye
bye
inde
go
take
drama
elsewher
:)
ff
readi
weekend
:)
smile
work
:)
:)
mood
bipolar
thank
:)
hahahahahahahahahahahahahaha
die
:)
liter
front
like
yoohoo
shatter
record
bajrangibhaijaanstorm
superhappi
:D
who'
awak
:-)
happi
friday
:-)
pleasur
doll
thank
wonder
energi
class
:)
f
:)
delight
m'dear
:)
hi
emma
:-)
ask
bellybutton
inni
outi
followback
beauti
girl
today
:)
lyka
followback
:)
alrd
joy
:)
ourdaughtersourprid
dhan
dhan
satguru
tera
hi
aasra
...
mani
congratul


great
day
hunni
:)
good
night
moon
:)
pandora
2
ap
studi
hall
haha
great
:)
follow
everyon
back
teamfollowback
:-)
ha
almost
gave
fuck
:)
boy
trash
wast
time
that'
sure
amaz
pic
today
ye
alreadi
ad
pleas
check
final
design
:)
noth
much
need
grow
:)
easi
clever
interest
audienc
welcom
:)
great
video
:D
ask
end
..
i'm
friend
:)
thank'
follow
keven
:)
ha
love
pic
clearli
much
learn
popular
english
pastim
:)
ashramcal
return
:p
catch
fall
fall
ontrack
:)
alway
make
thing
better
:)
❤
️
guess
they'll
show
german
also
subtitl
:)
pinter
:-)
goood
morninggg
jummah
mubarak
everyon
bless
amaz
day
ahead
:)
cute
:-)
🐶
stat
day
arriv
1
new
follow
unfollow
:)
via
fabul
time
miss
:)
xx
noth
perfect
smile
:)
never
show
much
:)
:-)
cheer
pete
listen
later
☺
☺
that'
build
sunday
:p
direct
attent
...
awesome-o
:)
get
us
tune
well
app
multipl
option
:D
engin
:)
happi
friday
:D
cya
week
:D
happi
shop
:)
fantast
show
harrog
home
gift
back
offic
2
day
jet
see
wonder
supplier
:)
nice
one
:D
ye
let'
:)
7
:)
:)


ok
good
night
wish
troy
ugli
met
today
:)
):
ok
today
fun
i'm
excit
tmrw
inde
:-)
:)
good
afternoon
:D
twitterfollowerswhatsup
happyfriedday
keepsafealway
loveyeah
emojasp_her
okay
:)
make
two
us
:D
hurt
read
people'
holiday
work
:D
nice
hear
want
switch
us
:)
pro
kit
would
best
choic
follow
link
beauti
...
winter
like
summer
russia
:)
thought
’
like
mate
:)
i'm
play
brain
dot
braindot
stat
week
arriv
1
new
follow
unfollow
:)
via
dm
:p
also
come
game
:)
thank
:)
:)
hope
enjoy
ok
first
time
chat
made
joke
lol
believ
wont
forget
u
:)
name
mymonsoonstori
bodi
shop
bodi
mist
vanilla
work
wonder
skin
:)
:)
best
youtub
...
keep
good
work
man
starsquad
sidemen
yaaayyy
best
friendaaa
shout
like
minion
:)
thank
ad
us
list
:)
make
sure
keep
touch
news
light
bulb
made
corn
wast
1tbps4
wow
prize
delight
eye
pick
win
burst
sing
ps4
mine
feel
divin
:)
bath
:)
even
’
wheeli
bin
nowher
put
get
readi
...
ubericecream
messengerforaday
:)
yeap
:)
cool
...
congrat
kyli
u
look
nice
:-)
thought
work
season


dieback
music
pack
total
worth
awesom
:)
dota
2
morn
:)
think
might
endit
london
photo
vid
minecon
london
video
:)
look
rememb
great
time
ok
...
sere
2
play
yah
min
..
haha
:)
joerin
scene
:)
joshan
power
tandem
:D
can't
sleep
need
tri
lay
bed
bore
stat
day
arriv
6
new
follow
unfollow
:)
via
b
call
friend
need
:)
...
hello
ligao
citi
albay
:-)
concert
bcyc
lnh
goodnight
love
luke
heart
:-)
love
💜
happi
birthday
enjoy
vacat
:)
congratul
sat
3rd
honorari
degre
everyon
alac
:)
hey
raspberri
island
skelo_ghost
:)
want
get
iph
0ne
6
free
kindli
check
bi0
thx
thank
u
:)
thank
great
:)
❤
️
😘
deserv
madadagdagan
pa
yan
:-)
congrat
:-)
bmc
<3
save
money
use
good
way
money
money
better
life
fun
:)
nice
day
11:11
u
find
ur
friend
ditch
u
caus
lot
thing
:)
wtf
done
im
actual
nice
person
order
dress
..
:)
:)
like
eye
:D
dear
person
pleas
studi
embarrass
urself
entropi
work
100
w
evolut
:)
ye
stay
touch
:)
instagram
look
forward
alway
love
giggl
loop
two
great
act
:)
cheer
mention
...
even
wrong
sec

matt
would
say
welcom
adulthood
...
:)
could
say
egg
face
:-)
help
...
stop
cri
:(
otl
nevermind
:(
least
got
jeon
soon
tweet
plant
claw
thigh
traction
zoom
away
:(
damnit
:(
use
pri
pv
...
wish
could
reliv
day
becom
nyc
pv
buy
way
commun
nyc
usa
klm
:(
realli
hot
:-(
monday
:(
go
stop
breakfast
earli
might
want
remov
11am
websit
even
mcd'
pull
trick
:(
mean
way
:(
3rd
load
hung
:(
wtf
suppos
without
headach
strike
:(
english
weather
need
fix
:(
live
fam
bam
cough
:(
absolut
gut
jame
bay
ticket
sold
manchest
:(
seventh
spot
na
lang
:(
otwolgrandtrail
splendour
:(
omg
swedish
hair
metal
legend
häirførc
studio
leav
nooo
:(
givecodpieceach
sore
alic
:(
i'm
readi
work
yet
:(
someon
unfav
:(
stile
:(
funni
mo
:(
someon
explain
sandrabland
:(
dont
sad
:(
ili
go
spend
night
pragu
leav
tomorrow
:(
one
friend
follow
littl
heart
attack
im
sorri
your
block
:(
sadi
i'm
annoy
:(
consid
lucki
favourit
charact
even
make
season
final
100
:(
boo
alreadi
turn
ye
915
:(
itun
i'm
wait
notanapolog
:(
layou

pretti
:(
:(
matt
hello
info
possibl
interest
jonatha
close
join
beti
:(
saludo
hey
someon
text
:(
anoth
dissapoint
review
:(
😫
<--
face
everi
guy
pull
get
head
feel
bae'
teeth
dick
:(
ahhh
well
:(
wanna
see
paper
town
w
:(
get
long
last
black
pump
:(
omfg
beauti
place
ever
miss
much
:(
hate
pimpl
:(
almost
12
i'm
tire
:(
honey
brown
sugar
good
ur
lip
:/
amount
stuff
use
face
could'v
use
food
instead
:(
ikr
hai
would
someon
sell
preciou
thing
:(
i'm
tire
:(
got
good
sleep
yea
man
...
suppos
footbal
tdi
man
plan
fail
:(
pleas
god
wanna
go
work
:(
kinda
miss
pamela
good
anna
:(
pleas
woe
can't
run
six
though
live
mexico
:(
idk
i'm
get
mini
bu
idk
time
i'm
go
get
:(
hi
harri
sorri
:(
we'r
work
mast
area
caus
intermitt
servic
work
complet
look
fun
snapchat
janniecam
snapchat
kikm
webcam
snapchatm
kikhorni
musicbiz
hotmusicdeloco
:(
braxton
gone
pro
:(
feel
bad
urban
unpreced
brainwash
see
tebow
alex
smith
wat
okaaay
huhu
:(
sayanggg
find
yah
phone
1
taxi
servic
2
second
schedul
blog
post
3

poor
must
tire
:(
ur
crazi
af
hurt
much
bc
u
know
shit
aint
real
edit
:(
forev
miss
love
lolo
:(
xbsbabnb
poor
:(
although
mum
ask
watch
w
…
stilll
:(
read
yet
she'd
ask
question
middl
movi
work
5
everyday
exhaust
:(
end
state
last
night
:(
:(
:(
:(
tbh
prefer
apma
14
idk
man
there
someth
right
year
apma
:(
u
give
350
ill
give
u
350
350
rt
anyth
pl
:(
lmao
back
get
realli
realli
seriou
...
:(
wonder
happen
earlier
realiz
leadership
:-(
wakeupgop
ah
dude
:(
wanna
take
adam
candi
eat
:(
pleas
watch
infinite'
mv
:(
can't
believ
ill
go
work
today
wish
:(
:(
hate
school
u
dont
understand
miss
lot
blame
school
like
can't
actual
put
pressur
ankl
hop
around
hous
lost
balanc
fell
:(
bull
shark
:(
late
trafficcc
miss
oscar
:(
need
translat
:(
last
time
one
work
:(
fb
gone
check
long
time
lol
asian
ummm
mayb
ur
film
child
pornographi
slutsham
fake
suicid
ect
:-(
poland
faraway
germani
girl
:(
still
believ
one
day
meet
good
quit
full
time
job
last
week
can't
find
anoth
one
:(
:(
well
cooki
better


omg
:(
everi
time
would
complain
skin
mom
everyon
get
pimpl
like
although
letsfootbal
atk
greymind
43
break
news
chri
gayl
say
cricket
2-3
month
due
back
surgeri
:(
cplt
20
cp
…
miss
hannah
montana
that'
raven
sooo
much
:-(
want
rt
:(
gonna
okay
love
:(
awh
:-(
💙
stay
💪
last
full
night
greec
:(
opu
inner
pleasur
disappoint
hear
especi
invest
one
:-(
ye
:(
bbq
attend
work
:(
three
190
row
33
:(
throat
hurt
bad
:(
lolliv
:(
im
watch
disney
channel
:(
nobodi
pick
one
right
:(
someon
gift
:(
calibraskaep
need
bad
lol
got
split
hard
lol
singl
singl
split
collat
spilt
singl
singl
spilt
singl
split
singl
singl
collat
:(
hope
feel
better
jay
:(
ye
mani
time
:(
quitkarwaoyaaro
stu
deacti̇v
:(
disappoint
:(
never
ever
lucki
handl
jabongatpumaurbanstamped
hello
need
2.5
g2a
code
pl
:(
thesi
thesi
thesi
thesi
thesi
thesi
thesi
thesi
thesi
thesi
sherep
nemen
ng
behey
ke
:(
deactiv
acc
:(
otwolgrandtrail
still
sad
kick
epic
motherfuck
:(
realli
want
anoth
tattoo
:(
someon
get
mad
reec
vm
tomorrow
pl

#### Expected Output: 
```0.9950```  
Pretty good!

# Part 5: Error Analysis

In this part you will see some tweets that your model misclassified. Why do you think the misclassifications happened? Specifically what kind of tweets does your model misclassify?

In [59]:
# Some error analysis done for you
print('Label Predicted Tweet')
for x,y in zip(test_x,test_y):
    y_hat = predict_tweet(x, freqs, theta)

    if np.abs(y - (y_hat > 0.5)) > 0:
        print('THE TWEET IS:', x)
        print('THE PROCESSED TWEET IS:', process_tweet(x))
        print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))

Label Predicted Tweet
bro
u
wan
cut
hair
anot
ur
hair
long
liao
bo
sinc
ord
liao
take
easi
lor
treat
save
leav
longer
:)
bro
lol
sibei
xialan
back
thnx
god
i'm
happi
:)
thought
ear
malfunct
thank
good
clear
one
apolog
:-)
stuck
centr
right
clown
right
joker
left
...
:)
happi
friday
:-)
follow
:)
x
teenchoic
choiceinternationalartist
superjunior
fight
oppa
:D
birthday
today
birthday
wish
hope
there'
good
news
ben
soon
:-)
good
morn
:-)
friday
󾰀
plan
day
current
play
shop
...
happi
friday
:)
3
good
nigth
:)
estoy
escuchando
enemi
god
actual
bye
bye
inde
go
take
drama
elsewher
:)
ff
readi
weekend
:)
smile
work
:)
:)
mood
bipolar
thank
:)
hahahahahahahahahahahahahaha
die
:)
liter
front
like
yoohoo
shatter
record
bajrangibhaijaanstorm
superhappi
:D
who'
awak
:-)
happi
friday
:-)
pleasur
doll
thank
wonder
energi
class
:)
f
:)
delight
m'dear
:)
hi
emma
:-)
ask
bellybutton
inni
outi
followback
beauti
girl
today
:)
lyka
followback
:)
alrd
joy
:)
ourdaughtersourprid
dhan
dhan
satguru
tera
hi
aas

wow
realli
hate
:)
watch
saw
dan
:)
thank
:-)
ilysm
xx
thank
trade
:)
look
forward
:-)
ks
lesley
:)
want
protect
relationship
other
i'll
talk
ppl
need
know
awesom
:)
oh
dang
zen
look
outsid
space
neat
:-)
look
good
outfit
:)
hiya
i'm
afraid
we'v
alreadi
pick
rider
year
we'll
look
next
year
:)
good
morn
u
:)
ha
ha
ok
honey
go
ahead
fyi
still
dig
ya
enjoy
vote
watch
side
:)
lucki
:)
learn
new
skill
take
time
perfect
master
👱
🏽
👽
🌳
😂
😂
😂
i'v
talk
u
coupl
time
u
seem
nice
funni
:)
agre
suspici
mind
:)
good
afternoon
:)
whoop
whoop
bring
fun
:)
new
report
talk
burn
calori
cold
work
harder
warm
feel
better
weather
:p
say
someth
i'm
give
:)
i'm
sorri
get
:D
good
night
:)
there'
gonna
anoth
one
final
:)
welcom
jessica
enjoy
:)
best
friend
carina
one
san
francisco
let
know
want
get
touch
:)
thank
mommi
teret
:)
:)
potassium
rehydr
drinkitallup
thirstquench
i'm
sure
tapir
calf
one
like
adventur
mealtim
:)
uhc
box
larg
scale
one
network
server
:p
ever
areal
we'll
lunch
dinner
:)
here'
one
extreme

thought'
kepler
452b
:D
progress
stone
transfer
chelsea
:D
gg
bro
:)
fun
seri
alway
part
part
defenit
...
♬
:p
cours
man
:)
nofx
shit
friskyfiday
yipee
:)
x
morn
kim
sorri
hear
shed
light
issu
look
:)
helen
great
incent
get
kid
eat
fruit
vege
:-)
welcom
sir
:)
know
averag
take
6
second
read
god
dam
mother
fuck
tweet
:)
thank
favourit
tweet
marin
:)
get
close
can't
wait
new
season
:)
gz
:D
name
morn
day
:-)
guess
sleep
list
thing
today
:-)
hai
rajeev
..
hope
u
r
hvng
funfil
friday.it
ws
reali
amaz
c
diff
side
kabir.fel
luv
u
:)
smile
pl
prettier
:)
mine
:)
happi
birthday
best
wish
dresden
germani
:)
love
<3
<3
<3
<3
pleas
share
wanna
loser
like
:)
♥
zayn
pleas
follow
would
mean
world
birthday
:D
love
much
amaz
<3
congratul
friend
graduat
lose
plot
:D
0
sleep
offici
way
pari
cannot
wait
:)
pari
tdf
2015
🇫
🇷
🍷
🍸
☀
️
🇫
🇷
🚲
🇫
🇷
🍷
🍸
☀
️
🚲
put
trust
:)
hey
good
suggest
guy
make
minion
slot
would
fun
cute
:)
happi
b'day
:)
hah
...
thousand
lie
:D
wow
look
good
isabella
make
wanna
start
work
ba

soon
base
london
i'm
person
workout
eat
healthi
minut
gain
weight
:-)
gm
meet
preciou
soft
ladi
real
friend
mizz
preciou
:)
;)
mani
like
...
aameen
..
:)
long
live
pakistan
..
bleedgreen
..
✌
🏻
️
alreadi
follow
back
oppa
:)
liam
sophia
chicago
:)
♥
btw
honeymoon
:)
yeah
bring
boom
da'esh
co-ord
fsa
estat
:)
welcom
:)
when'
next
meet
singer
dusti
tunisia
<3
music
tunisia
<3
wonder
nice
peopl
:-)
<3
youtub
...
la
class'
:p
hahaha
go
doug
:-)
forgot
ask
may
post
fb
happi
friday
:-)
visit
blog
everyon
fuckin
irrit
:)
also
mean
imma
go
back
twitter
activ
:D
caus
know
everyon
miss
;)
xd
hello
ladi
apolog
delay
pleas
dm
us
address
size
detail
we'll
ship
prize
:)
free
voucher
twitter
friend
use
fiverr
:)
oh
gina
busi
:-)
can't
wait
see
hear
xxx
soproud
happi
friday
everyon
hope
fantast
week
:)
friday
ye
one
team
:)
happi
belat
birthday
sweeti
:)
morn
emma
:)
thank
guy
:)
would
say
:-)
love
make
two
heart
one
:)
thank
ad
us
list
:)
make
sure
keep
touch
news
light
bulb
made
corn
wast
thank
mr
be

that'
great
idea
love
fellow
veggi
:-)
c
75
made
collagen
an
2win
hopetowin
:)
awww
thank
hey
million
:)
enjoy
weekend
hey
guess
need
build
content
around
imag
inventori
littl
bit
:)
gamer
follow
train
follow
retweet
follow
retweet
gain
activ
follow
:)
1
snap
:)
bring
togeth
friend
famili
loveforfood
foodforthought
thoughtfortheday
:)
carp
diem
mr
nath
:)
yep
i'd
without
ning
recommend
book
thought
wow
:)
think
gener
posit
lead
ty
appreci
recent
retweet
great
friday
:)
11:11
meet
michael
hug
tight
talk
tell
he'
import
love
much
make
smile
:D
long
feel
comfort
im
gonna
wear
want
mother
haha
...
sound
nice
...
:-)
congratul
daughter
:D
goodnight
love
everyon
hate
stupid
:)
thank
fill
although
opinion
still
stand
agre
disagre
harm
done
:)
see
saturday
:p
i'll
see
stormi
:D
time
2
parti
:D
upgrad
an
sync
plu
done
singl
remot
devic
get
folk
:)
haha
know
mess
:D
want
say
huge
thank
ff
thank
support
:)
found
song
mention
nylon
gvb
cd
take
mountain.titl
unto
other
theworldwouldchang
:)
relat
w

wanna
go
six
flag
:(
sorri
:(
shop
fun
:(
miss
cat
:(
miss
:(
would
like
abl
work
full
time
stuff
move
i'm
consid
take
fulltim
job
:(
yeah
i'v
seen
someth
tinder
fb
root
:(
woman
frustrat
awkward
food
kitchen
money
wallet
thank
barcelona
dad
home
month
beet
juic
either
:(
dci
today
wish
go
:-(
work
freez
:(
srsli
u
:(
hate
see
granddad
like
:(
sorri
pre
:(
want
minion
bucket
pleas
:(
yesterday
:(
fair
love
war
kapan
updat
:(
oh
ya
udah
dihapu
hilang
dari
muka
bumi
want
read
someon
give
link
😢
girl
quick
:(
mani
nasti
narrow
mind
peopl
:(
want
room
:-(
friend
respect
life
:-(
i'm
sorri
sadli
:(
suck
i'm
gona
miss
chello
parti
:(
hello
info
possibl
interest
jonatha
close
join
beti
:(
can't
watch
phone
sorri
:(
thank
still
<3
taxi
driver
thought
sophi
work
nintendo
took
us
gate
:(
secur
guard
gave
us
dirtiest
look
haha
omg
:-(
love
much
monica
see
soon
aaahhh
:(
crepe
40
—
kick
bae
:(
haha
sound
like
’
fun
game
:(
follow
:(
school
monday
wanna
meet
new
peopl
:(
...
havent
finish
yet
:(
u


i'm
sad
:(
cant
find
:(
oh
:(
build
suit
wanna
cut
k
i'm
friend
threw
sleepov
parti
:(
i'm
pretend
ok
like
usual
leav
alon
:(
pray
khami
...
:(
...
damn
forgot
hot
chocol
:(
i'm
crave
oreo
milk
:(
love
island
finish
feel
like
summer
gone
:(
miss
:(
push
client
way
host
mayb
better
wait
till
next
week
:(
im
irk
:(
someon
say
thank
u
goodby
chri
tomorrow
pl
:(
need
nate
ruess
album
:(
could
realli
use
hug
mayb
icecream
:(
lot
good
comic
event
seem
kid
:(
part
school
program
look
like
dog
food
...
:(
ugh
miss
bad
:(
aigoo
babi
:(
want
watch
paper
town
:-(
wanna
recov
:(
:-(
poor
boy
u
guy
togeth
:(
saw
one
doujin
think
killua
get
fuck
ging
bit
sad
caus
didnt
show
dick
hope
ginggon
:(
comput
broken
wait
fix
miss
much
twitter
guys.al
love
..
:(
feel
like
ytd
:(
joe
i'm
sick
come
round
make
soup
:(
cri
:(
pdapaghimok
stori
life
:(
haha
look
who'
talk
work
:(
smh
anyon
awak
:-(
ya
sorri
:-(
friend
:(
use
flexibl
:(
taxi
driver
go
realli
fast
i'm
scare
:(
bet
wonder
happen
:(
sheet
hurt
nanama

notic
z
peopl
wish
birthday
:(
:(
world
aw
let
feel
like
complet
shit
:(
uh
huh
:(
two
time
i'm
super
clever
like
:(
bitch
realli
went
fave
unfollow
:(
😩
💖
realli
want
visit
iceland
:(
zayn_come_back_we_miss_y
<3
<3
:(
much
starv
:-(
bc
feel
creepi
today
pmsl
mianh
milkeu
:(
wanna
meet
boy
:(
internet
total
bitch
lrt
bambam
like
soda
:(
meant
:(
1
:(
wish
could
go
shop
whole
weekend
taken
:(
annoy
peopl
usual
stay
asleep
wtf
:(
cheer
:-(
payback
like
video
tri
help
get
kit
kat
:(
i'm
even
tire
:-(
one
come
smoke
:(
87000
jobe
deliveri
pl
:(
happi
birthday
be
enjoy
day
beb
love
miss
sooo
muchi
:(
see
soon
sana
💞
🎈
🎂
🎉
😘
cuti
:(
alway
fall
asleep
earli
i'm
text
someon
:(
say
icecream
avail
:(
ask
go
bathroom
:-(
i'm
realli
enjoy
show
though
lagg
much
certain
peopl
:(
need
clear
head
:(
babi
went
back
sleep
i'm
bore
:-(
ughh
got
noth
pain
:(
laper
banget
:(
naw
:(
deep
dream
novel
hope
there'd
invis
page
beast
scuttl
around
there'
worm
track
bauuukkk
:(
bad
guy
cant
attend
today'
music
ba

:(
world
alterni
kill
realli
sad
leav
place
everyon
die
everyon
gonna
talk
abt
rat
boy
today
bc
:(
earth
assum
rain
london
like
influenc
overal
warm
dri
past
week
continent
europ
:(
rememb
fab
four
24
hour
call
damn
miss
much
:(
😀
🔫
r
u
sure
u
want
b
r
u
h
bare
surviv
w
tank
top
:(
thirsti
:(
konami
polici
pe
pc
version
happen
version
:(
sound
effect
:(
blame
ranti
:(
ummm
ok
new
develop
:(
guy
never
:(
—
aw
sorri
we'r
realli
bust
atm
..
shall
back
soon
bad
:(
like
video
mani
pervers
first
time
go
school
without
bracelet
:(
feel
odd
can't
fall
back
asleep
:(
im
twin
follow
back
bylfnnz
:(
unfair
..
u
ban
film
pakistan
:(
want
one
:(
feel
sick
:(
hmmm
10
min
get
train
i'm
current
15
min
away
:(
failsatlif
i'm
hungri
:(
want
birthday
alreadi
:(
complet
agre
press
:(
im
super
duper
tire
:(
bore
time
:(
know
...
ill
soon
promis
:(
waaah
wanna
chang
avi
usanel
:(
puppi
broke
foot
:(
where'
jaebum
babi
pictur
:(
mr
ahmad
maslan
cook
:(
hull
support
expect
misser
week
:-(


Later in this specialization, we will see how we can use deep learning to improve the prediction performance.

# Part 6: Predict with your own tweet

In [61]:
# Feel free to change the tweet below
my_tweet = 'This is a ridiculously bright movie. The plot was terrible and I was sad until the ending!'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['ridicul', 'bright', 'movi', 'plot', 'terribl', 'sad', 'end']
ridicul
bright
movi
plot
terribl
sad
end
[[0.48165774]]
Negative sentiment
